In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [14]:
df1= pd.read_csv('data1',index_col=0)
df2= pd.read_csv('data2',index_col=0)
df3= pd.read_csv('data3',index_col=0)
df4= pd.read_csv('data4',index_col=0)
df5= pd.read_csv('data5',index_col=0)
df6= pd.read_csv('data6',index_col=0)
df7= pd.read_csv('data7',index_col=0)
df8= pd.read_csv('data8',index_col=0)

In [15]:
frames = [df1,df2,df3,df4,df5,df7,df8]
nashik = pd.concat(frames,ignore_index=True)
nashik.describe(include='all')

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft
count,5497,5484,5497,5497,5461,5496,4544,3985.000000
unique,1758,2791,2,2,61,1167,867,NaN
top,"Nashik Road, Nashik",Mr Sanjay,Apartment,old,2,Price on Request,18.53,NaN
freq,297,185,4323,3651,2487,953,116,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1109.826098
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,875.052599
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,735.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,975.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1295.000000


In [16]:
nashik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5497 entries, 0 to 5496
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          5497 non-null   object 
 1   owners           5484 non-null   object 
 2   housetype        5497 non-null   object 
 3   house_condition  5497 non-null   object 
 4   BHK              5461 non-null   object 
 5   price            5496 non-null   object 
 6   per_month_emi    4544 non-null   object 
 7   total_sqft       3985 non-null   float64
dtypes: float64(1), object(7)
memory usage: 343.7+ KB


In [17]:
nashik['BHK'] = pd.to_numeric(nashik['BHK'],errors = 'coerce')
nashik['price'] = pd.to_numeric(nashik['price'],errors = 'coerce')
nashik['per_month_emi'] = pd.to_numeric(nashik['per_month_emi'],errors = 'coerce')

In [18]:
nashik

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft
0,"Sheetal Vihar, Bhagwant Nagar, Dr.Homi Bhabha ...",Mahendra Kotwal,Apartment,old,3.0,75.00,39.72,1550.0
1,"Samraat Dream Citi, Samta Nagar, Nashik",Jiten Dadarkar,Apartment,old,2.0,41.00,21.71,1000.0
2,"Suryaprakash Apartment,Nashik Road, Nashik",Pankaj,Apartment,old,2.0,53.35,28.25,970.0
3,"Adishvar Residency,Nashik Road, Nashik",Saurav,Apartment,old,2.0,55.00,29.13,1000.0
4,"Nashik Road, Nashik",Vivek Pavanaskar,Apartment,old,2.0,27.00,14.30,853.0
...,...,...,...,...,...,...,...,...
5492,"Vasantdadanagar, Adgaon,Nashik",Geeta Buildcon,Independent_house,old,2.0,NaN,NaN,NaN
5493,"Samarth Nagar, Makhmalabad,Nashik",Atharvashree Buildmat Private Limited,Independent_house,old,2.0,34.32,18.17,NaN
5494,"Bhor Town Ship, Nashik",Shree Prakash Developers,Independent_house,old,NaN,NaN,NaN,NaN
5495,"Chandshi, Nashik",Nirman Group,Independent_house,old,NaN,NaN,NaN,NaN


In [19]:
nashik.describe(include='all')

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft
count,5497,5484,5497,5497,5334.000000,4543.000000,4537.000000,3985.000000
unique,1758,2791,2,2,NaN,NaN,NaN,NaN
top,"Nashik Road, Nashik",Mr Sanjay,Apartment,old,NaN,NaN,NaN,NaN
freq,297,185,4323,3651,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2.021466,55.231691,21.548497,1109.826098
std,NaN,NaN,NaN,NaN,0.916778,295.747432,24.443782,875.052599
min,NaN,NaN,NaN,NaN,1.000000,1.000000,1.050000,150.000000
25%,NaN,NaN,NaN,NaN,1.000000,21.000000,11.650000,735.000000
50%,NaN,NaN,NaN,NaN,2.000000,32.650000,18.000000,975.000000
75%,NaN,NaN,NaN,NaN,2.000000,48.050000,26.440000,1295.000000


In [20]:
nashik.isnull().sum()

address               0
owners               13
housetype             0
house_condition       0
BHK                 163
price               954
per_month_emi       960
total_sqft         1512
dtype: int64

In [21]:
nashik.dropna(axis =0,inplace=True)

In [23]:
nashik.reset_index(drop=True)

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft
0,"Sheetal Vihar, Bhagwant Nagar, Dr.Homi Bhabha ...",Mahendra Kotwal,Apartment,old,3.0,75.00,39.72,1550.0
1,"Samraat Dream Citi, Samta Nagar, Nashik",Jiten Dadarkar,Apartment,old,2.0,41.00,21.71,1000.0
2,"Suryaprakash Apartment,Nashik Road, Nashik",Pankaj,Apartment,old,2.0,53.35,28.25,970.0
3,"Adishvar Residency,Nashik Road, Nashik",Saurav,Apartment,old,2.0,55.00,29.13,1000.0
4,"Nashik Road, Nashik",Vivek Pavanaskar,Apartment,old,2.0,27.00,14.30,853.0
...,...,...,...,...,...,...,...,...
3854,"Gayakhe Colony, Nashik Road,Nashik",Ejaj Sayyad,Independent_house,old,2.0,55.00,29.13,1650.0
3855,"Balram Nagar, Nashik",Sajish Gangurde,Independent_house,old,4.0,45.00,23.83,1450.0
3856,"Nashik Road, Nashik",Ejaj Sayyad,Independent_house,old,3.0,57.51,30.45,1475.0
3857,"Omkar Construction,Damodar Nagar, Pathardi Pha...",Vinit Sawant,Independent_house,old,1.0,15.00,7.94,525.0


In [26]:
X= nashik[['BHK','total_sqft']]
target= nashik.price

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
        X, target, test_size=0.2, random_state=42 )

In [28]:
from sklearn import datasets, linear_model, metrics
reg = linear_model.LinearRegression()
ridge = linear_model.Ridge()
lasso = linear_model.Lasso()

In [30]:
lasso.fit(X_train,y_train)
lasso.score(X_test,y_test)

-0.8397245492354126